In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error

In [2]:
import read_file as rf
import split_data as spt
import data_preprocessing as proc

# Neighborhood подход

Данный подход основан на матрице похожести. Можно рассматривать похожесть пользователей или похожесть объектов. В данной задаче нам необходимо реализовать данный алгоритм, основанный на похожести фильмов. 
 
Neighborhood подход имеет очень высокую вычислительную сложность. Нужно по списку оценок пользователей построить таблицу похожести объектов или пользователей. Данные вычисления локально работают около часа. Поэтому рекомендовалось использовать AWS Amazon

В item-oriented методе данного подхода similarity-matrix считается по формуле:

$$ sim(i, j) = \frac{\sum_{u \in{U}}(r_{u, i} - \bar{r})(r_{u, j} - \bar{r})} 
{\sqrt{\sum_{u \in{U}}(r_{u, i} - \bar{r})^2} \sqrt{\sum_{u \in{U}}(r_{u, j} - \bar{r})^2}} $$

Аналогично выглядит формула для user-oriented подхода:

$$ sim(n, k) = \frac{\sum_{i \in{I}}(r_{i, n} - \bar{r})(r_{i, k} - \bar{r})} 
{\sqrt{\sum_{i \in{I}}(r_{i, n} - \bar{r})^2} \sqrt{\sum_{i \in{I}}(r_{i, k} - \bar{r})^2}} $$

### Считываем данные

In [3]:
df_users, df_movies, df_rates = rf.read_zipped_file('ml-1m.zip')

### Делим данные

In [4]:
%time df_splited = spt.get_train_test_split(df_rates, df_users.index)

CPU times: user 1min 1s, sys: 58.7 s, total: 2min
Wall time: 2min 3s


In [5]:
%%time
movies_ind = spt.get_new_indexes(df_splited, 'movie_id')
users_ind = spt.get_new_indexes(df_splited, 'user_id')

CPU times: user 280 ms, sys: 2.42 ms, total: 282 ms
Wall time: 282 ms


In [6]:
df_splited[0].to_csv('./data/train_dataset.csv', sep=':', index=None, header=None)

Для того, чтобы протестировать алгоритм, необходимо запустить его локально. Это можно сделать с помощью командной строки (как делала я). Или же с помощью следующего кода:

In [12]:
%%time

keys = []
values = []
from similarity_matrix import SimilarityCount
mr_job = SimilarityCount(args=['./data/train_dataset.csv'])
with mr_job.make_runner() as runner:
    runner.run()
    for line in runner.stream_output():
        key, value = mr_job.parse_output_line(line)
        keys.append(key)
        values.append(value)

Если запускать с помощью командной строки, то результат будет сразу выводиться в файл, это удобнее для сохранения информации. В командной строке я запускала следющей командой:
<center><i>python similarity_matrix.py <./data/train_dataset.csv> ./data/train_similarity.csv -r local </i></center>


Но есть и недостатки у этого метода -- приходится считывать из файла и парсить, то, что вывел mrjob, что и делает следующая функция

In [7]:
def get_similarity_matrix_from_file(file_name):
    x = []
    y = []
    value = []
    with open('./data/%s'%file_name, 'r') as f:
        for l in f:
            new = l.replace('"', '').split('\t')
            value.append(new[1].replace('\n', ''))
            x_new, y_new = new[0].split('|')
            x.append(x_new)
            y.append(y_new)
    x = np.array(x).astype(int)
    y = np.array(y).astype(int)
    value = np.array(value).astype(float)
    y_max = y.max()
    x_max = x.max()
    matrix = np.zeros((x_max, y_max))
    for i in range(x.shape[0]):
        matrix[x[i]-1, y[i]-1] = value[i]
    return matrix

In [8]:
def get_values_from_test(df):
    y_true = []
    users = []
    movies = []
    for ind in df.index:
        row = df.loc[ind]
        y_true.append(row['rating'])
        users.append(row['user_id'])
        movies.append(row['movie_id'])
    return np.array(y_true), np.array(users), np.array(movies)

In [39]:
def get_predict(matrix, df, users, movies, users_to_predict, movies_to_predict, N):
    adjacency = proc.to_adjacency_df(df, users, movies)
    y_pred = []
    for i in range(movies_to_predict.shape[0]):
        user_rates = adjacency.loc[users_to_predict[i], :].as_matrix()
        user_indexes = np.nonzero(user_rates)[0]
        movie_indexes = np.array(adjacency.columns[user_indexes]-1)
        sim = sim_matrix[movies_to_predict[i]-1, movie_indexes]
        sort = np.argsort(sim)
        user_indexes = user_indexes[sort][:N]
        sim = sim[sort][:N]
        
        rates = user_rates[user_indexes]
        sum_sim = np.sum(sim)
        if sum_sim == 0:
            result = rates.mean()
        else:
            result = np.sum(sim*rates) / np.sum(sim)
        y_pred.append(result)
    return y_pred

Нужно распарсить подсчитанные значения и посчитать рейтинг для каждого нового фильма по следующей формуле:

$$\hat{r_{u, i}} = \frac{\sum_{j:r_{u, j}\neq 0}sim(i, j)r_{u, j}}{\sum_{j:r_{u, j}\neq 0}sim(i, j)}$$ 

Если бы использовался user-oriented метод, то формула была бы такой:

$$\hat{r_{u, i}} = \frac{\sum_{v:r_{v, i}\neq 0}sim(u, v)r_{v, i}}{\sum_{v:r_{v, i}\neq 0}sim(u, v)}$$

In [10]:
sim_matrix = get_similarity_matrix_from_file('train_similarity.csv')

Получим истинную метку и список пользователей и фильмов, оценки которых нужно предсказать

In [11]:
%time y_true, users_to_pred, movies_to_pred = get_values_from_test(df_splited[1])

CPU times: user 32.7 s, sys: 166 ms, total: 32.8 s
Wall time: 33 s


А теперь проанализируем, как количество фильмов максимальной схожести влияют на результат

In [40]:
N = np.array([1, 5, 10, 15])
loss = []
for i in range(N.shape[0]):
    y_pred = get_predict(sim_matrix, df_splited[1], users_ind[1], movies_ind[1], 
                         users_to_pred, movies_to_pred, N[i])
    loss.append(mean_squared_error(y_true=y_true, y_pred=y_pred))

In [41]:
loss

[1.9450583104059749, 1.2048315956785485, 1.1288446130221279, 1.109552275437957]

In [43]:
N = np.array([50, 100, 200, 400])
loss = []
for i in range(N.shape[0]):
    y_pred = get_predict(sim_matrix, df_splited[1], users_ind[1], movies_ind[1], 
                         users_to_pred, movies_to_pred, N[i])
    loss.append(mean_squared_error(y_true=y_true, y_pred=y_pred))

In [45]:
loss

[1.0569690371699407,
 0.98889703416120633,
 0.93832748586618586,
 0.93180653681235515]

| N size  | MSE  |
|---|---|
| 1  | 1.9450583104059749  |
|  15 |  1.109552275437957 |
|  100 | 0.98889703416120633  |
|400|0.93180653681235515|

Как видно из этой таблицы -- чем больше значени N, тем лучше результат

In [94]:
%time y_pred = get_predict(sim_matrix, df_splited[1], users_ind[1], movies_ind[1], users_to_pred, movies_to_pred)

CPU times: user 1min 25s, sys: 940 ms, total: 1min 26s
Wall time: 1min 28s


In [95]:
loss = mean_squared_error(y_true=y_true, y_pred=y_pred)

In [96]:
loss

0.9316676690692125

### Выводы

Данный подход хорош тем, что не нужно придумывать различные признаки. Нам не нужно знать о пользователях и фильмах ничего, кроме того, какиеоценки пользователи ставят каким фильмам.


Лучше хранить всю таблицу, но если очень мало места на диске, то можно ограничиться хранением лишь 400 схожих фильмов.

Достоинства:
    - Нет необходимости во вспомогательной информации о пользователях
Недостатки:
    - Требует много времени для вычисления
    - В моей реализации он оказался немного хуже, чем content-based